In [12]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','양천구','강서구']

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests; from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys



def address_to_latitude(address):
    global lat
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(urlparse(url).geturl(),
                     headers = {"Authorization": "KakaoAK 96d7359a39d27a12d4ccddf4bf82f85b"}) # Rest API
    json_obj = result.json()
    for document in json_obj['documents']:
        lat = document['y']
    return lat

def address_to_longtitude(address):
    global long
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    result = requests.get(urlparse(url).geturl(),
                     headers = {"Authorization": "KakaoAK 96d7359a39d27a12d4ccddf4bf82f85b"})
    json_obj = result.json()
    for document in json_obj['documents']:
         long = document['x']
    return long


results = []

for find in gu_list : 
    time.sleep(10)

    driver = webdriver.Chrome(
        executable_path= "chromedriver.exe"
    )
    url = "https://map.kakao.com/"
    driver.get(url)
    action = ActionChains(driver)
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(find + '애견동반식당')  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element_by_id("info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element_by_id("info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while True: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        time.sleep(1)
        # place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements_by_css_selector("#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element_by_xpath(xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            time.sleep(3)
            place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                
                if len(place_name) == 0:
                    continue # 광고
                name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                address = store_info.select('.info_item > .addr > p')[0].text
                latitude = address_to_latitude(address)
                longtitude = address_to_longtitude(address)
                results.append((name, address, latitude, longtitude))

            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element_by_id("info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

results

import pandas as pd
data_frame = pd.DataFrame(results, columns=['name', 'address',"lat","long"])
data_frame

5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
Arrow is Disabled
End of Crawl
5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
Arrow is Disabled
End of Crawl
4 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
End of Page
Arrow is Disabled
End of Crawl
5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
Arrow is Disabled
End of Crawl
2 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
End of Page
Arrow is Disabled
End of Crawl
5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
2 개의 페이지 있음
1  of  [  2  ] 
2  of  [  2  ] 
End of Page
Arrow is Disabled
End of Crawl
5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
Arrow is Disabled
End of Crawl
5 개의 페이지 있음
1  of  [  1  ] 
2  of  [  1  ] 
3  of  [  1  ] 
4  of  [  1  ] 
5  of  [  1  ] 
Arrow is Disabled
End of Crawl


[('38도씨식당', '서울 마포구 와우산로17길 19-17 2층', '37.5494298799352', '126.921615197284'),
 ('데이비드식당', '서울 마포구 와우산로 162-4 1층', '37.5551894481249', '126.930740637202'),
 ('땡스오트 연남점', '서울 마포구 성미산로23길 68', '37.5662739481646', '126.919897441386'),
 ('라오삐약', '서울 마포구 희우정로10길 5 1층', '37.5527328101125', '126.90443785635'),
 ('8810리스트레토바',
  '서울 마포구 월드컵북로6길 88-10',
  '37.5598656024226',
  '126.922997285783'),
 ('씨푸드살롱마레스타', '서울 마포구 월드컵북로4길 15 1층', '37.5572057386981', '126.920637926827'),
 ('얼띵앤키친', '서울 마포구 토정로5길 22 1층', '37.546414251193', '126.918572775584'),
 ('베이스캠프', '서울 마포구 잔다리로3안길 21', '37.5507854480578', '126.919761471991'),
 ('훌라훌라', '서울 마포구 희우정로10길 4', '37.5526332685366', '126.904683899821'),
 ('지하102호', '서울 마포구 양화로6길 57-22', '37.5493872950732', '126.918079087989'),
 ('바온하우스', '서울 마포구 성미산로17길 62', '37.564226548362', '126.917575120245'),
 ('유니드마이요거트', '서울 마포구 성미산로29길 44 1층', '37.5658536631345', '126.921959404'),
 ('루나씨엘로', '서울 마포구 동교로34길 21', '37.5608539329308', '126.925568658019'),
 ('아이노테이블', '서울

In [14]:
# 중복 삭제
data_frame.drop_duplicates(ignore_index = True, inplace = True)
data_frame

,name,address,lat,long
0,38도씨식당,서울 마포구 와우산로17길 19-17 2층,37.5494298799352,126.921615197284
1,데이비드식당,서울 마포구 와우산로 162-4 1층,37.5551894481249,126.930740637202
2,땡스오트 연남점,서울 마포구 성미산로23길 68,37.5662739481646,126.919897441386
3,라오삐약,서울 마포구 희우정로10길 5 1층,37.5527328101125,126.90443785635
4,8810리스트레토바,서울 마포구 월드컵북로6길 88-10,37.5598656024226,126.922997285783
...,...,...,...,...
200,효뜨,"서울 용산구 한강대로40가길 6 1,2층",37.5652702034365,126.924633178184
201,와일드그릴,경기 부천시 소사로 549,37.5067730532228,126.797233251004
202,여월가든,경기 부천시 원미로281번길 123,37.5116313720025,126.792574126688
203,뽀독 소사점,경기 부천시 소삼로 47-1 소사역푸르지오 상가125호,37.4794820091274,126.796899892434


In [15]:
# 저장
data_frame.to_csv('./data/pet_restaurant.csv')